Let's build a recommendation system using the movie **tags** to build user/movie profiles
1. We need to build a profile for each movie (simply by using tfidf vectorizer)
2. We need to build a profile for each user (average tfidf per movie watched by the user)

In [ ]:
import pandas as pd

In [ ]:
movies = pd.read_csv('../ml-latest-small/movies.csv')
tags = pd.read_csv('../ml-latest-small/tags.csv')

In [ ]:
movies

In [ ]:
movie_tags = movies.merge(tags, on="movieId").drop(['title','genres','userId','timestamp'],axis=1).groupby("movieId").apply(lambda x: ' '.join(x.tag)).reset_index()
movie_tags.columns = ['movieId','doc']

In [ ]:
tf_idf.transform(movie_tags[movie_tags.movieId==1].doc)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_idf = TfidfVectorizer()

In [ ]:
X = tf_idf.fit_transform(movie_tags.doc)
X

In [ ]:
ratings = pd.read_csv('../ml-latest-small/ratings.csv')
user_movie = ratings[['userId', 'movieId']].merge(movie_tags,on="movieId").drop('doc',axis=1).drop_duplicates(['userId', 'movieId']).sort_values('userId')
user_movie

In [ ]:
import numpy as np
from tqdm import tqdm
user_profile = {}
user_movie_count = {}

for rec in tqdm(user_movie.itertuples(), total=len(user_movie)):
    user_profile[rec.userId] = user_profile.get(rec.userId, np.zeros(1744)) + tf_idf.transform(movie_tags[movie_tags.movieId==rec.movieId].doc)
    user_movie_count[rec.userId] = user_movie_count.get(rec.userId, 0) + 1
    
    

In [ ]:
user_profile_normalize = {}
for userId in user_profile:
    user_profile_normalize[userId] = (user_profile[userId] / user_movie_count[userId]).A1

In [ ]:
user_profile_normalize[1]

In [ ]:
[tf_idf.get_feature_names()[i] for i in user_profile_normalize[12].argsort()[-10:]]